In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

# Load the pre-trained ResNet101 model without the fully connected layers
resnet_base = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in resnet_base.layers:
    layer.trainable = False

# Create a new model on top of the ResNet101 base
x = Flatten()(resnet_base.output)
x = Dense(1024, activation='tanh')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='tanh')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

model = Model(resnet_base.input, output)

171446536/171446536 [==============================] - 1s 0us/step


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [5]:
from keras.models import Model
flatten = Flatten()
new_layer2 = Dense(4, activation='softmax', name='my_dense_2')

inp2 = model.input
out2 = new_layer2(flatten(model.output))

model2 = Model(inp2, out2)
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [6]:
model.compile(
    optimizer="adam",
    loss='sparse_categorical_crossentropy',
    metrics=['acc']
)

In [7]:
import pathlib
data_dir = "/content/drive/MyDrive/Hepatoma-Staging-Project/Dataset/Hepatoma-Stages-Dataset/ADASYN_images"
#data_dir = tf.keras.utils.get_file('data', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [8]:
batch_size = 32
img_height = 224
img_width = 224

In [9]:
import tensorflow as tf
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=32)

Found 683 files belonging to 4 classes.
Using 547 files for training.


In [10]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=32)

Found 683 files belonging to 4 classes.
Using 136 files for validation.


In [13]:
model.fit(x=train_ds,
           validation_data=val_ds,
    epochs=10,
    verbose=1
)

Epoch 1/10
18/18 [==============================] - 5s 235ms/step - loss: 1.3865 - acc: 0.4826 - val_loss: 0.8098 - val_acc: 0.6471
Epoch 2/10
18/18 [==============================] - 5s 245ms/step - loss: 1.2572 - acc: 0.5265 - val_loss: 0.7779 - val_acc: 0.6103
Epoch 3/10
18/18 [==============================] - 5s 239ms/step - loss: 1.1873 - acc: 0.5576 - val_loss: 0.7410 - val_acc: 0.7721
Epoch 4/10
18/18 [==============================] - 5s 254ms/step - loss: 1.2128 - acc: 0.5466 - val_loss: 0.7885 - val_acc: 0.6765
Epoch 5/10
18/18 [==============================] - 5s 239ms/step - loss: 1.2201 - acc: 0.5375 - val_loss: 0.9083 - val_acc: 0.6912
Epoch 6/10
18/18 [==============================] - 5s 239ms/step - loss: 1.2228 - acc: 0.5082 - val_loss: 0.8137 - val_acc: 0.6471
Epoch 7/10
18/18 [==============================] - 5s 245ms/step - loss: 1.2112 - acc: 0.5320 - val_loss: 0.8327 - val_acc: 0.6324
Epoch 8/10
18/18 [==============================] - 5s 237ms/step - loss: 1.

In [12]:
model.evaluate(val_ds)

5/5 [==============================] - 1s 140ms/step - loss: 1.0209 - acc: 0.4779


[1.0209276676177979, 0.4779411852359772]

10th epoch: Epoch 10/10
18/18 [==============================] - 5s 242ms/step - loss: 1.3483 - acc: 0.4899 - val_loss: 1.0209 - val_acc: 0.4779 \\
20th epoch: Epoch 10/10
18/18 [==============================] - 5s 235ms/step - loss: 1.1857 - acc: 0.5229 - val_loss: 0.9645 - val_acc: 0.5294

